In [1]:
import numpy as np
import astropy.io.fits as pyfits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord

import aplpy
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams, style
from matplotlib.colors import ListedColormap

from sys import argv
import warnings
import time
import os

/usr/local/lib/python3.7/site-packages/mpl_toolkits/axes_grid/__init__.py:12: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid module was deprecated in Matplotlib 2.1 and will be removed two minor releases later. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist provies the same functionality instead.
  obj_type='module')


In [2]:
#---------------------------------------
# Input some parameters from parameter.ipynb
#---------------------------------------
%run parameter.ipynb


#---------------------------------------
# Input parameters used in this program:

###-------------------------------------
### Path name & outname & imageName
# path_image, path_BT, path_figure, path_samping, path_region
# outname_b4,  outname_b7
# BT_B4, BT_B7


In [3]:
#----------------------------------------
# define the functions
#----------------------------------------

def do_convert_unit(filename_in, filename_out, convert_unit):

    '''
    Purpose: 
    Convert the fitsfile unit from Jy/beam to uJy/beam or mJy/beam 
    
    Input:
    - filename_in           # Input filename [str]
    - filename_out          # Output filename [str]
    - convert_unit          # Convert unit: 'Jy2uJy' or 'Jy2mJy'
    
    Output:
      N/A
    '''   
    
    # Get the data & the header
    im = pyfits.getdata(filename_in)
    hd = pyfits.getheader(filename_in)
    hd_unit_in = hd['BUNIT']    
    
    # Set the parameters based on the input convert unit
    if convert_unit == 'Jy2mJy':
        convert_value = 1e3
        hd_unit_out = 'mJy/beam'
    elif convert_unit == 'Jy2uJy':
        convert_unit = 1e6
        hd_unit_out = 'uJy/beam'
    
    # Convert the images unit
    im_convert = im*convert_value
    pyfits.writeto(filename_out, im_convert, hd, overwrite=True)
    
    # Change the header of the output fitsfile
    pyfits.setval(filename_out, 'BUNIT', value = hd_unit_out)
    
    print ('Convert images %s to %s.'%(filename_in, filename_out))
    print ('Unit from %s to %s.'%(hd_unit_in, hd_unit_out))
    print ('#----------------------------------------')

#----------------------------------------------------------
#Fitsfile = do_convert_unit(outname, convert_unit, stoke)

In [4]:
%%time
#----------------------------------------
# mask the images with PI < 3 sigma_PI
#----------------------------------------
outnames = [
            outname_b4, outname_b7
            ]

for outname in outnames:
  
    #----------------------------------------
    # Get the Imagename
    I_imageName, Q_imageName, U_imageName, PI_imageName, PI_cut3PI_imageName,\
    PA_imageName, Per_imageName, Per_cut3PI_imageName, BT_imageName = slt_imageName(outname)
    
    # Get the rms and criteria
    rms_I, rms_Q, rms_PI, I_clip, PI_lowclip, PI_highclip, \
    sampling, PI_vmax, convert_unit = slt_parm(outname)
    
    # Get the FITS file and the header
    I_data = pyfits.getdata(I_imageName)
    I_hd = pyfits.getheader(I_imageName)
    PI_data = pyfits.getdata(PI_imageName)
    PI_hd = pyfits.getheader(PI_imageName)
    Per_data = pyfits.getdata(Per_imageName)
    Per_hd = pyfits.getheader(Per_imageName)

    #----------------------------------------
    # Per (PI > 3 rms_PI)
    Per_data_mask = Per_data
    Per_data_mask[PI_data < 3.* rms_PI] = np.nan
    Per_data_mask[I_data < 3.* rms_I] = np.nan
    
    # PI (PI > 3 rms_PI)
    PI_data_mask = PI_data
    PI_data_mask[PI_data < 3.* rms_PI] = np.nan
    PI_data_mask[I_data < 3.* rms_I] = np.nan
    
    # Output the fits files
    pyfits.writeto(Per_cut3PI_imageName, Per_data_mask, Per_hd, overwrite=True)
    pyfits.writeto(PI_cut3PI_imageName, PI_data_mask, PI_hd, overwrite=True)
    
#----------------------------------------------------------

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in less
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in less


CPU times: user 308 ms, sys: 634 ms, total: 942 ms
Wall time: 1.1 s


In [5]:
%%time
#----------------------------------------
# Call the convert unit function

outnames = [
            outname_b4, outname_b7
            ]

for outname in outnames:
    
    #PI_cut3PI_imageName  = '%s%s_PI_cut3PI.image.fits'%(path_image, outname)

    filename_in  = '%s%s_PI_cut3PI.image.fits'%(path_image, outname)
    filename_out = '%s%s_PI_cut3PI_Jy2mJy.image.fits'%(path_image, outname)
    convert_value = 'Jy2mJy'

    do_convert_unit(filename_in, filename_out, convert_value)
    
#----------------------------------------------------------

Convert images ../Image_Data/n1333iras4a_B4_uv24_2900kl_sm200_100mas_noshift_PI_cut3PI.image.fits to ../Image_Data/n1333iras4a_B4_uv24_2900kl_sm200_100mas_noshift_PI_cut3PI_Jy2mJy.image.fits.
Unit from Jy/beam to mJy/beam.
#----------------------------------------
Convert images ../Image_Data/n1333iras4a_B7_uv24_2900kl_sm200_100mas_noshift_PI_cut3PI.image.fits to ../Image_Data/n1333iras4a_B7_uv24_2900kl_sm200_100mas_noshift_PI_cut3PI_Jy2mJy.image.fits.
Unit from Jy/beam to mJy/beam.
#----------------------------------------
CPU times: user 146 ms, sys: 296 ms, total: 442 ms
Wall time: 505 ms
